In [225]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing, cross_validation, svm 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline

In [226]:
path = os.getcwd() + '/TestResults.csv'

data = pd.read_csv(path, header =0, usecols=list(range(0,19)),names=['ShotID','Shooter', 'X (in)', 'Y (in)', 'Distance From Target', 'Notes', 'Accel X Max','Accel X Min','Accel Y Max','Accel Y Min','Accel Z Max','Accel Z Min','1 Sec Vol before shot x','1 Sec Vol before shot y','1 Sec Vol before shot z','Avg 1 sec Vol Before Shot','Data Direction X','Data Direction Y','Data Direction Z'])
data.head(10)

,ShotID,Shooter,X (in),Y (in),Distance From Target,Notes,Accel X Max,Accel X Min,Accel Y Max,Accel Y Min,Accel Z Max,Accel Z Min,1 Sec Vol before shot x,1 Sec Vol before shot y,1 Sec Vol before shot z,Avg 1 sec Vol Before Shot,Data Direction X,Data Direction Y,Data Direction Z
0,A1,Rob,0.5,-4,4.031129,NaN,7.796,-26.868,2.284,-30.617,23.592,-35.396,0.023,0.087,0.144,0.084667,Backwards,Up,Right
1,A2,Rob,0.75,-1.25,1.457738,NaN,5.406,-22.477,21.088,-19.005,34.754,-91.866,0.009,0.102,0.072,0.061000,Backwards,Down,Right
2,A3,Rob,1,-3.5,3.640055,NaN,12.129,-23.401,4.118,-25.848,44.724,-36.157,0.018,0.106,0.066,0.063333,Backwards,Up,Left
3,A4,Rob,0,0,0.000000,NaN,16.477,-15.787,17.942,-86.024,44.647,-10.233,0.020,0.121,0.052,0.064333,Forward,Up,Left
4,A5,Rob,0,-0.5,0.500000,NaN,11.875,-20.743,5.909,-70.595,28.496,-28.328,0.034,0.079,0.101,0.071333,Backwards,Up,Left
5,A6,Rob,0.75,-2,2.136001,NaN,13.297,-19.767,5.631,-59.218,30.593,-47.645,0.020,0.063,0.026,0.036333,Backwards,Up,Right
6,A7,Rob,-1,-2.25,2.462214,NaN,13.748,-26.394,36.392,-26.336,57.025,-78.563,0.006,0.019,0.081,0.035333,Backwards,Down,Right
7,A8,Rob,1,-0.5,1.118034,NaN,13.810,-28.625,33.169,-23.276,61.148,-77.764,0.014,0.112,0.055,0.060333,Backwards,Down,Right
8,A9,Rob,1,-3,3.162278,NaN,34.390,-18.842,23.789,-43.258,53.841,-11.875,0.048,0.118,0.147,0.104333,Forward,Up,Left
9,B1,Rob,-3,-6.5,7.158911,NaN,11.813,-38.154,2.926,-20.044,54.473,-57.365,0.018,0.080,0.349,0.149000,Backwards,Up,Right


In [227]:
data = data[['X (in)', 'Y (in)', 'Distance From Target', 'Accel X Max', 'Accel X Min','Accel Y Max','Accel Y Min','Accel Z Max','Accel Z Min']]

In [228]:
data.dropna(inplace=True)#data.fillna(-9999,inplace=True)
data.describe()

,Distance From Target,Accel X Max,Accel X Min,Accel Y Max,Accel Y Min,Accel Z Max,Accel Z Min
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,2.646765,19.766211,-20.817974,19.947434,-42.939947,55.302342,-39.599118
std,1.944539,15.550466,8.738517,13.452989,20.573652,40.952996,38.065828
min,0.000000,1.748000,-56.072000,-0.493000,-96.592000,9.845000,-153.483000
25%,1.118034,10.274500,-26.449000,7.446000,-52.007750,28.300750,-54.524750
50%,2.243034,15.763000,-18.919000,18.909500,-39.638500,40.476000,-25.922000
75%,3.923051,25.783000,-14.650000,30.470000,-26.726500,67.941250,-11.630750
max,9.013878,102.817000,-9.246000,52.620000,-11.564000,156.902000,-1.595000


In [229]:
# data['X_Change'] = data['Accel X Max'] - data['Accel X Min']
# data['Y_Change'] = data['Accel Y Max'] - data['Accel Y Min']
# data['Z_Change'] = data['Accel Z Max'] - data['Accel Z Min']

In [230]:
#data = data[['Distance From Target', 'X_Change', 'Y_Change', 'Z_Change']]

In [231]:
forecast_col = 'Distance From Target'
forecast_cols = ['X (in)', 'Y (in)']

In [232]:
data.describe()

,Distance From Target,Accel X Max,Accel X Min,Accel Y Max,Accel Y Min,Accel Z Max,Accel Z Min
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,2.646765,19.766211,-20.817974,19.947434,-42.939947,55.302342,-39.599118
std,1.944539,15.550466,8.738517,13.452989,20.573652,40.952996,38.065828
min,0.000000,1.748000,-56.072000,-0.493000,-96.592000,9.845000,-153.483000
25%,1.118034,10.274500,-26.449000,7.446000,-52.007750,28.300750,-54.524750
50%,2.243034,15.763000,-18.919000,18.909500,-39.638500,40.476000,-25.922000
75%,3.923051,25.783000,-14.650000,30.470000,-26.726500,67.941250,-11.630750
max,9.013878,102.817000,-9.246000,52.620000,-11.564000,156.902000,-1.595000


In [233]:
X = np.array(data.drop([forecast_col],1))
y = np.array(data[forecast_col])

In [255]:
X.shape, y.shape

((76, 8), (76,))

In [235]:
X = preprocessing.scale(X)

/home/altock/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [246]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.17)

In [247]:
clf = svm.SVR(kernel='rbf')
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
accuracy

0.68360027182164318

In [210]:
path = os.getcwd() + '/TestResults.csv'

data = pd.read_csv(path, header =0, usecols=list(range(0,19)),names=['ShotID','Shooter', 'X (in)', 'Y (in)', 'Distance From Target', 'Notes', 'Accel X Max','Accel X Min','Accel Y Max','Accel Y Min','Accel Z Max','Accel Z Min','1 Sec Vol before shot x','1 Sec Vol before shot y','1 Sec Vol before shot z','Avg 1 sec Vol Before Shot','Data Direction X','Data Direction Y','Data Direction Z'])

In [211]:
data = data[['X (in)', 'Y (in)', 'Accel X Max', 'Accel X Min','Accel Y Max','Accel Y Min','Accel Z Max','Accel Z Min']]
data.dropna(inplace=True)
data = data[data['X (in)'] != 'Missing']
data = data[data['Y (in)'] != 'Missing']
data.columns
data.describe()

,Accel X Max,Accel X Min,Accel Y Max,Accel Y Min,Accel Z Max,Accel Z Min
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,19.766211,-20.817974,19.947434,-42.939947,55.302342,-39.599118
std,15.550466,8.738517,13.452989,20.573652,40.952996,38.065828
min,1.748000,-56.072000,-0.493000,-96.592000,9.845000,-153.483000
25%,10.274500,-26.449000,7.446000,-52.007750,28.300750,-54.524750
50%,15.763000,-18.919000,18.909500,-39.638500,40.476000,-25.922000
75%,25.783000,-14.650000,30.470000,-26.726500,67.941250,-11.630750
max,102.817000,-9.246000,52.620000,-11.564000,156.902000,-1.595000


In [212]:
X = np.array(data.drop(forecast_cols,1))
y = np.array(data[forecast_cols[1]])


In [213]:
X = preprocessing.scale(X)

In [214]:
y

array(['-4', '-1.25', '-3.5', '0', '-0.5', '-2', '-2.25', '-0.5', '-3',
       '-6.5', '-4.5', '-1', '-5.5', '-1.5', '-4.25', '-3.5', '-2.25',
       '-8.5', '-7.5', '-1.75', '-3', '-2', '-5.5', '-0.75', '-0.25',
       '-0.75', '-0.25', '-0.5', '-6.5', '-3.5', '-2.25', '-2', '-4.25',
       '1.75', '-1.75', '1.25', '1.25', '0.5', '-1', '3.5', '0', '-0.5',
       '-0.25', '-3.25', '-3.25', '-4.25', '-4', '0.75', '-3.25', '-0.25',
       '-1.75', '1.25', '0', '-1.75', '-1.25', '-1', '2.75', '5', '4.25',
       '0', '-1', '0', '0', '-0.5', '0.5', '0.5', '-0.25', '-1.5', '0',
       '-2', '0', '0.25', '-2', '-2.75', '0', '2.5'], dtype=object)

In [215]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.1)

In [216]:
clf = LinearRegression() # svm.SVR(kernel='rbf')
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
accuracy

0.32024655416080694

In [256]:
from sklearn import gaussian_process